In [23]:
!pip freeze > requirements.txt

In [3]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   - -------------------------------------- 1.0/39.0 MB 10.1 MB/s eta 0:00:04
   -- ------------------------------------- 2.9/39.0 MB 7.0 MB/s eta 0:00:06
   ---- ----------------------------------- 4.5/39.0 MB 7.7 MB/s eta 0:00:05
   ------ --------------------------------- 6.0/39.0 MB 7.4 MB/s eta 0:00:05
   ------ --------------------------------- 6.6/39.0 MB 7.2 MB/s eta 0:00:05
   ------ --------------------------------- 6.6/39.0 MB 7.2 MB/s eta 0:00:05
   ------- -------------------------------- 7.1/39.0 MB 5.0 MB/s eta 0:00:07
   -------- ------------------------------- 7.9/39.0 MB 4.7 MB/s eta 0:00:07
   -------- ------------------------------- 8.7/39.0 MB 4.5 MB/s eta 0:00:07
   --------- ------------------------------ 9.2/39.0 MB 4.4 MB/s eta 0:00:07
   --------- ------------------------------ 9.7/39.0 MB 4.1 MB/s eta 0:00:08
   -


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\shiva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\shiva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.9 MB 5.0 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.9 MB 5.3 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.9 MB 5.4 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/9.9 MB 5.5 MB/s eta 0:00:01
   ---------------------- ----------------- 5.5/9.9 MB 5.6 MB/s eta 0:00:01
   --------------------------- ------------ 6.8/9.9 MB 5.6 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/9.9 MB 5.6 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/9.9 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   - --------------------

In [3]:
import torch
from transformers import pipeline
from accelerate.test_utils.testing import get_backend
# automatically detects the underlying device type (CUDA, CPU, XPU, MPS, etc.)
DEVICE, _, _ = get_backend()
pipe = pipeline(task="image-feature-extraction", model_name="google/vit-base-patch16-384", device=DEVICE, pool=True)

C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to google/vit-base-patch16-224 and revision 3f49326 (https://huggingface.co/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [18]:
### Load reference image embeddings
import os
print(os.listdir("./reference_images"))
ref_embeddings = [(img_path, pipe(f"reference_images/{img_path}")[0]) for img_path in os.listdir("./reference_images")]

['Ian.jpg', 'Reshma.jpg', 'Shiva.png']


In [43]:
cosine_similarity(torch.tensor(ref_embeddings[0][1]), torch.tensor(ref_embeddings[0][1]), dim=0)

tensor(1.)

In [ ]:
import cv2
from torch.nn.functional import cosine_similarity
ip_address = ''  # Replace with the IP address of your camera
port = ''                # Replace with the port number for your camera
username = ''          # Replace with the username for your camera
password = ''
url_640x480 = f"rtsp://{username}:{password}@{ip_address}:{port}/stream2"
url_1080p = f"rtsp://{username}:{password}@{ip_address}:{port}/stream1"
rtsp_url = url_640x480  # Set it to either `url_640x480` or `url_1080p` based on the desired resolution
# Open the RTSP stream
cap = cv2.VideoCapture(rtsp_url)
#cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# Check if the RTSP stream is opened successfully
try:
    if not cap.isOpened():
        print("Failed to open RTSP stream")

    while True:
        ret, frame = cap.read()  
        if not ret:
            print("Error: Could not read frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))  # Detect faces
        counter = 0
        if len(faces) > 0:
            
            for (x, y, w, h) in faces:
                cv2.imwrite(f"./data/tmp_img{counter}.png", frame[max(0,x-100):x+w+100, max(0, y-100):y+h+100])
                embedding = pipe(f"./data/tmp_img{counter}.png")
                similarity_score = [(ref_emb[0], cosine_similarity(torch.Tensor(embedding), torch.Tensor(ref_emb[1]), dim=1)) for ref_emb in ref_embeddings]
                matching_details = max(similarity_score, key=lambda x: x[1])
                
                counter += 1
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
                cv2.putText(frame, f"{matching_details[0].split(".")[0]}:{torch.round(matching_details[1][0]*100)}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2.imshow('Face Detection', frame)


        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the RTSP stream and close the window
except Exception as e:
    print(e)
finally:
    cap.release()
    cv2.destroyAllWindows()

In [60]:
torch.round(torch.tensor(0.12321)*100, decimals=0)

tensor(12.)